In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv("dataset/train_features.csv")
y_train = pd.read_csv("dataset/train_labels.csv")
X_test = pd.read_csv("dataset/valid_features.csv")
y_test = pd.read_csv("dataset/valid_labels.csv")

X_train = pd.DataFrame(X_train, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])
X_test = pd.DataFrame(X_test, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])

cleanup_nums = {"genre": 
                {"classic pop and rock": 1, 
                 "dance and electronica": 2,
                 "folk":3,
                 "jazz and blues":4,
                 "metal":5,
                 "pop":6,
                 "punk":7,
                 "soul and reggae":8
                }}
y_train.replace(cleanup_nums, inplace=True)
y_test.replace(cleanup_nums, inplace=True)


tfidf_vect = TfidfVectorizer()
X_title_cv_train = tfidf_vect.fit_transform(X_train['title'])
df_title_cv_train = pd.DataFrame(X_title_cv_train.toarray(), columns=tfidf_vect.get_feature_names())
X_train = pd.concat([X_train, df_title_cv_train], axis=1)
X_train = X_train.drop(["title"], 1)

X_title_cv_test = tfidf_vect.transform(X_test['title'])
df_title_cv_test = pd.DataFrame(X_title_cv_test.toarray(), columns=tfidf_vect.get_feature_names())
X_test = pd.concat([X_test, df_title_cv_test], axis=1)
X_test = X_test.drop(["title"], 1)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
#X_train.head(10)

In [2]:
lgr = MLPClassifier(random_state=1, max_iter=300)
lgr.fit(X_train,y_train["genre"])
lgr_pred = lgr.predict(X_test)

# let's how our model performed
print("Classificaion report: \n")
print(classification_report(y_test["genre"], lgr_pred))
print("Confusion matrix: \n")
print(confusion_matrix(y_test["genre"], lgr_pred))

Classificaion report: 

              precision    recall  f1-score   support

           1       0.18      0.33      0.23        55
           2       0.20      0.16      0.18        45
           3       0.34      0.39      0.36        64
           4       0.35      0.14      0.20        44
           5       0.54      0.30      0.39        66
           6       0.69      0.55      0.62        74
           7       0.45      0.59      0.51        44
           8       0.22      0.26      0.24        58

    accuracy                           0.35       450
   macro avg       0.37      0.34      0.34       450
weighted avg       0.39      0.35      0.36       450

Confusion matrix: 

[[18  7 10  4  1  3  1 11]
 [ 8  7  8  1  3  1  6 11]
 [18  3 25  3  3  1  2  9]
 [13  2 11  6  3  1  1  7]
 [ 9  6  4  0 20  4 13 10]
 [17  2  3  0  0 41  6  5]
 [ 7  2  3  1  1  4 26  0]
 [12  6 10  2  6  4  3 15]]


/home/wildan/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
